In [1]:
import os
from Bio import SeqIO
import gffutils
from collections import defaultdict

In [2]:
GENOME_PATH = "../datasets/schizosaccharomyces_pombe/GCF_000002945.2_ASM294v3_genomic.fna"
GFF_PATH = "../datasets/schizosaccharomyces_pombe/genomic.gff"
ORGANISM = "fission_yeast"

### Load Genome

In [3]:
print('loading genome...')
#reading entire genome into memory - this would NOT work if we use a big genome like humans
genome = SeqIO.to_dict(SeqIO.parse(GENOME_PATH, "fasta"))
print('done!')

loading genome...
done!


#### Genome exploration

In [4]:
genome

{'NC_003424.3': SeqRecord(seq=Seq('GATCACGTACATCACCTTGTAAGAATTTATCTGCAATAGTCCTTCGGTATTGTA...ATC'), id='NC_003424.3', name='NC_003424.3', description='NC_003424.3 Schizosaccharomyces pombe strain 972h- genome assembly, chromosome: I', dbxrefs=[]),
 'NC_003423.3': SeqRecord(seq=Seq('GATCTCGCAACTCTTAATAAAGCTAATTCCTGCTAATTCGCTATACACTAAATC...TTT'), id='NC_003423.3', name='NC_003423.3', description='NC_003423.3 Schizosaccharomyces pombe strain 972h- genome assembly, chromosome: II', dbxrefs=[]),
 'NC_003421.2': SeqRecord(seq=Seq('GATCAGCCAAAATGGCTGATCCAGCTATTTAGCAGGTTAAGGTCTCGTTCGTTA...GGG'), id='NC_003421.2', name='NC_003421.2', description='NC_003421.2 Schizosaccharomyces pombe strain 972h- genome assembly, chromosome: III', dbxrefs=[]),
 'NC_088682.1': SeqRecord(seq=Seq('AATGTGTAATTTGAATCAATTTCTATCCTTTTAAAAATACATCATAAAGTGTAC...TGT'), id='NC_088682.1', name='NC_088682.1', description='NC_088682.1 Schizosaccharomyces pombe isolate MT1 mitochondrion, complete genome', dbxrefs=[])}

### Build GFF DB

In [5]:
if not os.path.exists(f"{ORGANISM}_genome.db"):
    print("creating GFF database...")
    db = gffutils.create_db(
        GFF_PATH,
        dbfn=f"{ORGANISM}_genome.db",
        force=True,
        keep_order=True,
        merge_strategy="merge",
        sort_attribute_values=True
    )
else:
    print("loading existing GFF database...")
    db = gffutils.FeatureDB(f"{ORGANISM}_genome.db", keep_order=True)
print("done!")

loading existing GFF database...
done!


#### DB Exploration

In [6]:
print([f for f in db.featuretypes()])

['CDS', 'LONG_TERMINAL_REPEAT', 'RNase_P_RNA', 'exon', 'five_prime_UTR', 'gene', 'intron', 'lnc_RNA', 'mRNA', 'ncRNA', 'pseudogene', 'rRNA', 'region', 'sequence_feature', 'snRNA', 'snoRNA', 'tRNA', 'three_prime_UTR']


In [7]:
feat = next(db.features_of_type('CDS'))
print(feat)
feature_sequence = genome[feat.seqid].seq[feat.start - 1 : feat.end]
print(feature_sequence)
print(f'{len(feature_sequence) == (feat.end - feat.start + 1)}')


NC_003424.3	RefSeq	CDS	1	5662	.	-	0	ID=cds-NP_595040.1;Dbxref=GeneID:2541932,GenBank:NP_595040.1,PomBase:SPAC212.11;Name=NP_595040.1;gbkey=CDS;gene=tlh1;locus_tag=SPOM_SPAC212.11;partial=true;start_range=.,1;Parent=rna-NM_001018168.1;product=RecQ type DNA helicase;protein_id=NP_595040.1
GATCACGTACATCACCTTGTAAGAATTTATCTGCAATAGTCCTTCGGTATTGTACATTGTTCCAAGCATAGTAAACTAACGATATCAAGTTTGCCTTTCTAGCCCATGACCTACAGTCAGAAGTGTAAGCCATATCACTGTCGGCATGTTCAAACTTTGTCAAACcacaaaataaacaaagtcCTTGAAATCGAATACGTAGTTTACATTCTCGCAAGTTGTGGTCGGCCTTGCCACATTTATAACAAGTAGATAAGCGTACGGGGCATGCTTTCCCGGTATGAGCACGAATTTCTGTGTCTGGGTTACCAAGAGTGCAACTTAGACATTCATCTTTATACACTCGAAAGAATCGCTCGAGTTGTGCTGAAATGTCAGTTGAGTCTACCCATTGTTTTTTGACGGCTTGCACACGACTTTTATACTCGGCGAAATGAATGGTACGAGAATCAACATCACCATCACCATCACCATCACCAATACCAATACCAATACCAACAATACCACTCATACCTCCAACACTACCATGAACACGATTCATGTCATGGTTGACGCCCTGTTGTACATTGTTCAAATGTTCATCGTATGTGCGTTTATGACCACGACGATGGCTGAGCGACATGTCTTCCAAACCATACTTTAGATTCGTATTAGCATTGCCAAAGCTTTTCTTCCATGTTGGCGATACAGACGAAGGGAACAAAGTGATTTC

### Parse data from DB into .CSV

#### Get genome sequence and state sequence
For the purpose of this project, we define a coding sequence ONLY as a sequence labeled with `CDS` in the GFF file. Everything else we consider to be non-coding.

In [8]:
#get cds intervals
cds_intervals = defaultdict(list)

for cds in db.features_of_type('CDS'):
    cds_intervals[cds.seqid].append((cds.start, cds.end))

len(cds_intervals.keys())

4

In [9]:
#get coding and noncoding sequences
sequences = []
state_mask = []

for seqid, seqinfo in genome.items():
    chromosome = seqinfo.seq

    labels = [False] * len(chromosome)

    for start, end in cds_intervals[seqid]:
        for i in range(start - 1, end):
            labels[i] = True
    
    base_seq = "".join([b for b in chromosome])
    label_seq = "".join(['1' if c else '0' for c in labels])

    sequences.append(base_seq)
    state_mask.append(label_seq)

In [10]:
#entire genome in my memory
entire_genome_sequence = "".join(sequences).upper()
entire_state_sequence = "".join(state_mask).upper()
print(len(entire_genome_sequence) == len(entire_state_sequence))

True


#### Parse into k-mers and CSV
Will be using 3-mers but this can easily be changed

In [ ]:
k = 3
with open(f"{ORGANISM}_{k}-mers.csv", "w") as f:
    f.write("3mer,coding\n")
    for idx in range(len(entire_genome_sequence) - k + 1):
        #assuming state depends on middle nucleotide (this assumption is kinda whacky)
        #maybe later want to just discard k-mers at transitions between states but for now this is fine?
        f.write(f"{entire_genome_sequence[idx : idx + k]},{entire_state_sequence[idx + k//2]}\n")
